In [9]:
import pandas as pd
import numpy as np
from decimal import Decimal
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import json
import time
# from lemmatize import lemmatize
x_chat = chatGPT()
x_gemini = gemini()




In [50]:
df = pd.read_csv('../output/df_with_examples.csv')
df = df.dropna(subset=["Example1"])

In [51]:

job_cols = [[f"Example_Description{i}",f"Example_Job_Title{i}"] for i in range(1,4)]
for job in job_cols[0:]:
    for col in job:
        df = x_chat.run_batch_embeddings(df,col)

df.to_csv('../output/df_with_examples_embeddings.csv', index=False)

Processing final batch 0 to 10
Processing final batch 0 to 10
Processing final batch 0 to 10
Processing final batch 0 to 10
Processing final batch 0 to 10
Processing final batch 0 to 10


In [8]:
df_exp.description.iloc[2]

'Sales managers that determine price schedules and discount rates.'

In [29]:
# random_descriptions = df_exp['description'].sample(10, random_state=2)
# print("\n".join(random_descriptions.values))


In [6]:
df_exp = pd.read_csv('../input/gpts_labels/gpt_exposure_embeddings.csv')

df_exp.task_embedding = df_exp.task_embedding.apply(lambda x: x.strip("[]").split(", "))

task_embeddings = []
for x in df_exp.task_embedding:
    task_embeddings.append([float(y) for y in x])
df_exp.task_embedding = task_embeddings


task_embeddings = np.array(task_embeddings)
task_embeddings = np.vstack(task_embeddings)


df_exp.title_embedding = df_exp.title_embedding.apply(lambda x: x.strip("[]").split(", "))
title_embeddings = []
for x in df_exp.title_embedding:
    title_embeddings.append([float(y) for y in x])
df_exp.title_embedding = title_embeddings


task_embeddings.shape

(11582, 3072)

In [ ]:
df = pd.read_csv('../output/df_with_examples_embeddings.csv')
df.loc[:, ["Job1_ONET", "Job2_ONET", "Job3_ONET"]] = 'N/A'

# list(df.Job1_embedding.apply(ast.literal_eval))
for i in range(3):
    df[f"Example_Description{i+1}_embedding"] = df[f"Example_Description{i+1}_embedding"].apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
    df[f"Example_Job_Title{i+1}_embedding"] = df[f"Example_Job_Title{i+1}_embedding"].apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])




In [56]:

results = []
# results = list(pd.read_csv("../output/onet_df.csv").values)

for index, row in list(df.iterrows())[0:]:
    print("***********************")
    print(f"Processing {index}")
    name = row["organization name"]
    print(name)
    print(row["generated_description"])
        
    job_results = []
    for i in range(3): 
        example = row[f"Example{i+1}"]
        description = row[f'Example_Description{i+1}']
        job_title = row[f'Example_Job_Title{i+1}']
        print(f"\nProcessing Example: {example} \nJob: {description}\n")

        job_embedding = np.array(row[f'Example_Description{i+1}_embedding']).reshape(1,-1)
        job_title_embedding = np.array(row[f'Example_Job_Title{i+1}_embedding']).reshape(1,-1)
        cosine_sim = cosine_similarity(job_embedding, task_embeddings, 'cosine')
        
        top3 = sorted(zip(cosine_sim[0], df_exp[['title', 'Task', 'description','title_embedding']].values), reverse=True)[:3]
        for cosine_sim_job, onet in top3:
            print(cosine_sim_job)
            onet_title_embedding = np.array([onet[3]])
            cosine_sim_title = cosine_similarity(job_title_embedding, onet_title_embedding, 'cosine')[0][0]
            print(f"{onet[2]} \nCosine Similarity: {cosine_sim_job}, Person Cosine Similarity: {cosine_sim_title}")
            results.append([name,example,row[f"situation{i+1}_conf_interval"], row[f"situation{i+1}_conf_interval_reasoning"],job,onet[0],onet[1],job_title, cosine_sim_job,cosine_sim_title])
 

    if index % 50 == 0:
        print("**Saving Results**\n\n")
        onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
        onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
        onet_df.to_csv("../output/onet_df.csv", index=False)
        time.sleep(10)
        
    print("***********************\n\n")


onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
onet_df.to_csv("../output/onet_df.csv", index=False)

***********************
Processing 0
Luming Labs
Luming Labs offers a cloud platform that leverages the power of GPUs to enable efficient management of AI models, machine learning, scientific research, and other computationally intensive tasks. Their goal is to make cloud computing accessible to everyone, fostering innovation across various fields.

Processing Example: A research scientist at a pharmaceutical company is working on developing a new drug to treat a rare disease. They need to run complex simulations and analyze vast amounts of data to identify promising drug candidates. 
Job: Research Scientists that conduct research and development activities in the fields of engineering, chemistry, physics, or biology.

0.6561886470592388
Natural sciences managers that plan or direct research, development, or production activities. 
Cosine Similarity: 0.6561886470592388, Person Cosine Similarity: 0.5832916665363581
0.642665368835281
Chemical engineers that conduct research to develop ne

In [55]:
onet_df.iloc[0]

organization_name                                                          Luming Labs
example                              A research scientist at a pharmaceutical compa...
situation_conf_interval                                                            9.0
situation_conf_interval_reasoning    The scientist needs to analyze large datasets ...
job                                         [Example_Description3, Example_Job_Title3]
onet_title                                                   Natural Sciences Managers
onet_task                            Plan or direct research, development, or produ...
example_job_title                                                  Research Scientists
task_similarity                                                               0.656189
job_title_similarity                                                          0.583292
Name: 0, dtype: object

In [4]:
# pd.read_csv("../output/onet_df.csv")


df[df["organization name"]=="Leveragai"]



,organization name,full_description,industries,headquarters location,founded date,description,CB rank,headquarters region,postal code,estimated revenue,...,situation3_conf_interval_reasoning,Job1_embedding,Job1_title_embedding,Job2_embedding,Job2_title_embedding,Job3_embedding,Job3_title_embedding,Job1_ONET,Job2_ONET,Job3_ONET
1446,Leveragai,Leveragai is an AI-driven company that special...,"Artificial Intelligence (AI), Consulting, Corp...","Baltimore, Maryland, United States",2023-06-01,"Artificial Intelligence, Consultancy, Solution...","468,809","Greater Baltimore-Maryland Area, East Coast, S...",21223,—,...,While Leveragai's AI could automate customer s...,"[-0.008085760287940502, -0.019565412774682045,...","[-0.0012607682729139924, -0.018306555226445198...","[-0.02003137767314911, -0.02099858596920967, -...","[0.015798592939972878, -0.0158467348664999, -0...","[-0.03139056637883186, -0.033278316259384155, ...","[-8.562059520045295e-05, -0.0230860598385334, ...",N/A,N/A,N/A


In [10]:
s1 = "Animators and Multimedia Artists that create storyboards to illustrate scenes or action sequences for films, television programs, or other media."
s2 = "Special effects artists and animators that apply story development, directing, cinematography, and editing to animation to create storyboards that show the flow of the animation and map out key scenes and characters."



In [13]:
e1 = x_chat.get_embedding(s1)
#e1 = df_exp[df_exp.description.str.contains("managers that evaluate the organization")].task_embedding.iloc[0]
e2 = x_chat.get_embedding(s2)

#e2 = x_chat.get_embedding("Computer Information Systems Managers evaluate cloud service offering different provider select suitable option company 's need")
e1 = np.array(e1).reshape(1,-1)
e2 = np.array(e2).reshape(1,-1)
cosine_similarity(e1,e2, 'cosine')[0][0]

0.6625274673336069

In [17]:
title_embeddings = np.array(df_exp.title_embedding)
title_embeddings = np.vstack(title_embeddings)

In [147]:
onet_df = pd.read_csv("../output/results v3/onet_df.csv")
onet_df = onet_df[[col for col in onet_df.columns if "titles" not in col]]
onet_df[["Job_similarity_titles"]]= 'N/A'
len(onet_df)

6093